In [1]:
import os
import json
from dotenv import load_dotenv
from  langchain_google_genai  import  ChatGoogleGenerativeAI
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain_openai import ChatOpenAI
from sentence_transformers import CrossEncoder

In [2]:
import pandas as pd
import numpy as np
import time
import logging

logging.basicConfig(filename='error_log.txt', level=logging.ERROR)

In [3]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

## Thực hiện gen câu trả lời từ GPT

In [4]:
def load_chroma_db(persist_dir):
    if not os.path.exists(persist_dir):
        raise FileNotFoundError(f"The directory '{persist_dir}' does not exist.")
    
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    )

    # Load the Chroma vector store
    vector_db = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
    
    print("Loading Chroma vector store successfully.")
    return vector_db

vector_db = load_chroma_db("vector_db")

Loading Chroma vector store successfully.


In [4]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# # Khởi tạo mô hình LLM với cấu hình
# llm = ChatOpenAI(
#     temperature=0.1,
#     streaming=True,
#     model="gpt-4o-mini",
#     openai_api_key=openai_api_key,
# )

# Dung api cua google
google_api = os.getenv("GOOGLE_API_KEY")
llm  =  ChatGoogleGenerativeAI ( 
            model = "gemini-2.0-flash-exp",
            google_api_key = google_api,
            temperature = 0.5,
            max_tokens = 2000,
)

system_prompt = """
You are an Intellectual Property Lawyer named iLaw. Your primary responsibility is to assist users with legal questions related to intellectual property law. 

### Context:
{context}

### Role and Guidelines:
1. **Role**: You are acting as a professional lawyer specializing in Intellectual Property Law.
2. **Language**: All responses must be in **Vietnamese**.
3. **Audience**: Users may not have prior knowledge of legal terms, so your responses should be:
   - Clear, detailed, and easy to understand.
   - Free of unnecessary jargon.
   - Supplemented with examples or analogies when necessary.
4. **Fallback**: If no relevant documentation is found, respond with: **"Chúng tôi không tìm thấy thông tin liên quan."**
5. **Prohibited**: Do not guess, assume, or provide advice that cannot be supported by laws or references.
---

### Instructions:
When answering, follow this structured reasoning process:
1. **Identify the user's question**: Determine the specific area of Intellectual Property Law being addressed.
2. **Explain the concept in detail**: Provide a step-by-step explanation of relevant legal terms or laws.
3. **Apply to user's scenario**: Illustrate how the laws or terms apply to the user's context.
4. **Provide actionable advice**: Suggest next steps, documents, or authorities the user should contact.

"""


In [5]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Question: {question}")
    ]
)

# Khởi tạo chuỗi RAG với prompt_template
rag_chain = (
    RunnableMap(
        {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    )
    | prompt_template
    | llm
    | StrOutputParser()
)

In [5]:
#  Kết hợp các retriever và mô hình reranker vào một ensemble retriever
class EnhancedEnsembleRetriever:
    def __init__(self, retrievers, weights, reranker_model=None):
        self.retrievers = retrievers
        self.weights = weights
        self.reranker = CrossEncoder(reranker_model) if reranker_model else None

    def invoke(self, query):
        # Lấy kết quả từ từng retriever
        all_results = []
        for retriever, weight in zip(self.retrievers, self.weights):
            results = retriever.get_relevant_documents(query)
            for doc in results:
                # Áp dụng trọng số cho từng tài liệu
                doc.metadata['score'] = doc.metadata.get('score', 1.0) * weight
            all_results.extend(results)

        # Loại bỏ trùng lặp 
        unique_results = []
        seen_content = set()

        for doc in all_results:
            content = doc.page_content
            if content not in seen_content:
                unique_results.append(doc)
                seen_content.add(content)
        
        # Rerank nếu có mô hình reranker
        if self.reranker:
            query_doc_pairs = [(query, doc.page_content) for doc in unique_results]
            scores = self.reranker.predict(query_doc_pairs)
            reranked_results = sorted(
                zip(unique_results, scores),
                key=lambda x: x[1], 
                reverse=True
            )
            return [doc for doc, score in reranked_results]

        # Nếu không rerank, trả về kết quả theo trọng số
        return sorted(unique_results, key=lambda x: x.metadata['score'], reverse=True)


In [8]:
# Hàm tạo retriever từ Chroma và BM25
def create_retriever(vector_db, query, k=4):
    # Tạo BM25 retriever
    chroma_retriever = vector_db.as_retriever(search_type='similarity', search_kwargs={'k': k})

    documents_bm25 = [
        Document(page_content=doc.page_content, metadata=doc.metadata)
        for doc in vector_db.similarity_search("", k=300)
    ]

    bm25_retriever = BM25Retriever.from_documents(documents_bm25)
    bm25_retriever.k = k

    # Kết hợp cả Chroma và BM25 vào một ensemble retriever
    ensemble_retriever = EnhancedEnsembleRetriever(
        retrievers=[chroma_retriever, bm25_retriever],
        weights=[0.7, 0.3],
        reranker_model='cross-encoder/ms-marco-MiniLM-L-12-v2'
    )

    return ensemble_retriever


# Retrieve documents từ vector database
def retrieve_documents(ensemble_retriever, query, top_k=4):
    docs = ensemble_retriever.invoke(query=query)
    return docs[:top_k]


# Định dạng documents thành chuỗi
def format_docs(docs):
    formatted_docs =""
    for i, doc in enumerate(docs):
        formatted_docs += f"Document {i+1}:\n{doc.page_content}\n\n"
        
    return formatted_docs

In [9]:
def generate_answer(vector_db, question, top_k=5):

    # Tạo retriever kết và truy xuất những tài liệu liên quan
    ensemble_retriever = create_retriever(vector_db, question, k=top_k)
    docs = retrieve_documents(ensemble_retriever, question, top_k=top_k)
    # Định dạng tài liệu thành chuỗi văn bản
    formatted_docs = format_docs(docs)

    # print("Tài liệu trả về: ", formatted_docs)
    # Gọi chuỗi RAG với ngữ cảnh đầy đủ
    output = rag_chain.invoke({"context": formatted_docs, "question": question})
    
    # Xử lý metadata của các tài liệu để lấy liên kết và tiêu đề
    seen_links = set()
    unique_links = []
    unique_titles = []

    for doc in docs:
        link = doc.metadata.get('link', 'no link')
        title = doc.metadata.get('title', 'no title')
        
        if link not in seen_links:
            seen_links.add(link)
            unique_links.append(link)
            unique_titles.append(title)
    return output, unique_links, unique_titles

In [6]:
def generate_answer_non_kb(question):

    formatted_docs =" Chúng tôi không có tài liệu cho bạn, hãy trả lời bằng tri thức của bạn"
    output = rag_chain.invoke({"context": formatted_docs, "question": question})
    return output

In [7]:
question = "Luật sở hữu trí tuệ ỏ Việt Nam gồm những luật gì?"
answer = generate_answer_non_kb(question)
print(answer)

Chào bạn, tôi là iLaw, luật sư chuyên về sở hữu trí tuệ. Rất vui được hỗ trợ bạn.

Về câu hỏi của bạn, luật sở hữu trí tuệ ở Việt Nam chủ yếu được quy định trong các văn bản pháp luật sau:

1.  **Luật Sở hữu trí tuệ năm 2005 (được sửa đổi, bổ sung năm 2009, 2019, 2022):** Đây là luật chính, bao quát hầu hết các khía cạnh của sở hữu trí tuệ, bao gồm:
    *   **Quyền tác giả và quyền liên quan:** Bảo hộ các tác phẩm văn học, nghệ thuật, khoa học, biểu diễn, bản ghi âm, ghi hình, chương trình phát sóng,...
    *   **Quyền sở hữu công nghiệp:** Bảo hộ sáng chế, kiểu dáng công nghiệp, nhãn hiệu, chỉ dẫn địa lý, tên thương mại, bí mật kinh doanh,...
    *   **Quyền đối với giống cây trồng:** Bảo hộ các giống cây trồng mới.
    
    Luật này quy định về các điều kiện bảo hộ, chủ sở hữu, quyền và nghĩa vụ của chủ sở hữu, các hành vi xâm phạm và chế tài xử lý vi phạm.

2.  **Các văn bản hướng dẫn thi hành:** Để cụ thể hóa các quy định của Luật Sở hữu trí tuệ, có nhiều nghị định, thông tư hướng 

## Gen câu trả lời

In [13]:
df = pd.read_csv("model_answer_comparison.csv")

In [14]:
df["gemini_non_kb"] = ''

In [17]:
# Dùng gemini mà không cung cấp tri thức
n = len(df)
for i in range(n):
    print(f"Processing question {i+1}/{n}")
    question = df.iloc[i]['question']
    try:
        # Sinh câu trả lời (không có KB)
        answer = generate_answer_non_kb(question)
        df.at[i, 'gemini_non_kb'] = answer
    except Exception as e:
        logging.error(f"Error at question {i+1}: {e}")
        print(f"Error: {e}")
        
    # Sau mỗi 10 câu hỏi, chờ lâu hơn để tránh bị rate-limit
    if i % 10 == 0:  
        time.sleep(5)

Processing question 1/400
Processing question 2/400
Processing question 3/400
Processing question 4/400
Processing question 5/400
Processing question 6/400
Processing question 7/400
Processing question 8/400
Processing question 9/400
Processing question 10/400
Processing question 11/400
Processing question 12/400
Processing question 13/400
Processing question 14/400
Processing question 15/400
Processing question 16/400
Processing question 17/400
Processing question 18/400
Processing question 19/400
Processing question 20/400
Processing question 21/400
Processing question 22/400
Processing question 23/400
Processing question 24/400
Processing question 25/400
Processing question 26/400
Processing question 27/400
Processing question 28/400
Processing question 29/400
Processing question 30/400
Processing question 31/400
Processing question 32/400
Processing question 33/400
Processing question 34/400
Processing question 35/400
Processing question 36/400
Processing question 37/400
Processing

,link,label,date,title,question,content,answer,gpt_non_kb,gemini_non_kb
0,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hiệu,23:00 | 27/11/2024,"Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu c...","Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu c...",Căn cứ khoản 6 Điều 93Luật Sở hữu trí tuệ 2005...,"Theo Luật Sở hữu trí tuệ Việt Nam, thời gian b...","Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ...",
1,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hiệu,23:00 | 27/11/2024,"Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu c...",Thủ tục gia hạn Giấy chứng nhận đăng ký nhãn h...,Theo Điều 31Nghị định 65/2023/NĐ-CPquy định th...,Thủ tục gia hạn Giấy chứng nhận đăng ký nhãn h...,"Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ...",
2,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hiệu,23:00 | 27/11/2024,"Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu c...",Tải Mẫu Tờ khai yêu cầu gia hạn hiệu lực Giấy ...,Mẫu Tờ khai yêu cầu gia hạn hiệu lực Giấy chứn...,Để tải mẫu tờ khai yêu cầu gia hạn hiệu lực Gi...,"Chào bạn,\n\nTôi là iLaw, luật sư chuyên về sở...",
3,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Quyền sở hữu công nghiệp,21:00 | 27/11/2024,Quyền sở hữu công nghiệp gồm các quyền nào?,Quyền sở hữu công nghiệp gồm các quyền nào?,Căn cứ khoản 4 Điều 4Luật Sở hữu trí tuệ 2005đ...,Quyền sở hữu công nghiệp là một phần quan trọn...,"Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ...",
4,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Quyền sở hữu công nghiệp,21:00 | 27/11/2024,Quyền sở hữu công nghiệp gồm các quyền nào?,Quyền sở hữu công nghiệp được xác lập như thế ...,Theo khoản 3 Điều 6Luật Sở hữu trí tuệ 2005đượ...,Quyền sở hữu công nghiệp tại Việt Nam được xác...,"Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ...",
...,...,...,...,...,...,...,...,...,...
395,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hàng hóa,21:10 | 25/10/2023,Bao bì thương phẩm là gì? Bao bì dùng để đựng ...,Bao bì dùng để đựng hàng hóa bán lẻ có được xe...,Căn cứ quy định Điều 3Thông tư 05/2019/TT-BKHC...,Để trả lời câu hỏi của bạn về việc bao bì dùng...,"Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ...",
396,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hàng hóa,21:10 | 25/10/2023,Bao bì thương phẩm là gì? Bao bì dùng để đựng ...,Vị trí nhãn hàng hóa đối với hàng hóa có cả ba...,Căn cứ quy định khoản 2 Điều 4Thông tư 05/2019...,"Theo pháp luật Việt Nam, vị trí nhãn hàng hóa ...","Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ...",
397,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Dịch vụ đánh giá công nghệ,09:00 | 25/10/2023,Mẫu Giấy chứng nhận đủ điều kiện hoạt động dịc...,Mẫu Giấy chứng nhận đủ điều kiện hoạt động dịc...,Dưới đây là mẫu Giấy chứng nhận đủ điều kiện h...,Chúng tôi không tìm thấy thông tin liên quan.,"Chào bạn,\n\nTôi là iLaw, luật sư chuyên về sở...",
398,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Dịch vụ đánh giá công nghệ,09:00 | 25/10/2023,Mẫu Giấy chứng nhận đủ điều kiện hoạt động dịc...,Trình tự cấp Giấy chứng nhận đủ điều kiện hoạt...,Căn cứ quy định Điều 34Nghị định 76/2018/NĐ-CP...,Chúng tôi không tìm thấy thông tin liên quan.,"Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ...",


In [19]:
df1 = pd.read_csv("model_answer_comparison.csv")
df1["gemini_non_kb"] = ''

In [20]:
df1["gemini_non_kb"]= df["gpt_non_kb"]

In [22]:
df1.to_csv("model_answer_comparison.csv", index=False)

In [23]:
df1

,link,label,date,title,question,content,answer,gpt_non_kb,gemini_non_kb
0,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hiệu,23:00 | 27/11/2024,"Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu c...","Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu c...",Căn cứ khoản 6 Điều 93Luật Sở hữu trí tuệ 2005...,"Theo Luật Sở hữu trí tuệ Việt Nam, thời gian b...",1. **Xác định câu hỏi của người dùng**: Người ...,"Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ..."
1,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hiệu,23:00 | 27/11/2024,"Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu c...",Thủ tục gia hạn Giấy chứng nhận đăng ký nhãn h...,Theo Điều 31Nghị định 65/2023/NĐ-CPquy định th...,Thủ tục gia hạn Giấy chứng nhận đăng ký nhãn h...,Để gia hạn Giấy chứng nhận đăng ký nhãn hiệu t...,"Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ..."
2,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hiệu,23:00 | 27/11/2024,"Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu c...",Tải Mẫu Tờ khai yêu cầu gia hạn hiệu lực Giấy ...,Mẫu Tờ khai yêu cầu gia hạn hiệu lực Giấy chứn...,Để tải mẫu tờ khai yêu cầu gia hạn hiệu lực Gi...,1. **Xác định câu hỏi của người dùng**: Người ...,"Chào bạn,\n\nTôi là iLaw, luật sư chuyên về sở..."
3,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Quyền sở hữu công nghiệp,21:00 | 27/11/2024,Quyền sở hữu công nghiệp gồm các quyền nào?,Quyền sở hữu công nghiệp gồm các quyền nào?,Căn cứ khoản 4 Điều 4Luật Sở hữu trí tuệ 2005đ...,Quyền sở hữu công nghiệp là một phần quan trọn...,1. **Xác định câu hỏi của người dùng**: Người ...,"Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ..."
4,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Quyền sở hữu công nghiệp,21:00 | 27/11/2024,Quyền sở hữu công nghiệp gồm các quyền nào?,Quyền sở hữu công nghiệp được xác lập như thế ...,Theo khoản 3 Điều 6Luật Sở hữu trí tuệ 2005đượ...,Quyền sở hữu công nghiệp tại Việt Nam được xác...,1. **Xác định câu hỏi của người dùng**: Người ...,"Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ..."
...,...,...,...,...,...,...,...,...,...
395,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hàng hóa,21:10 | 25/10/2023,Bao bì thương phẩm là gì? Bao bì dùng để đựng ...,Bao bì dùng để đựng hàng hóa bán lẻ có được xe...,Căn cứ quy định Điều 3Thông tư 05/2019/TT-BKHC...,Để trả lời câu hỏi của bạn về việc bao bì dùng...,Bao bì dùng để đựng hàng hóa bán lẻ có thể đượ...,"Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ..."
396,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hàng hóa,21:10 | 25/10/2023,Bao bì thương phẩm là gì? Bao bì dùng để đựng ...,Vị trí nhãn hàng hóa đối với hàng hóa có cả ba...,Căn cứ quy định khoản 2 Điều 4Thông tư 05/2019...,"Theo pháp luật Việt Nam, vị trí nhãn hàng hóa ...",Vị trí nhãn hàng hóa đối với hàng hóa có cả ba...,"Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ..."
397,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Dịch vụ đánh giá công nghệ,09:00 | 25/10/2023,Mẫu Giấy chứng nhận đủ điều kiện hoạt động dịc...,Mẫu Giấy chứng nhận đủ điều kiện hoạt động dịc...,Dưới đây là mẫu Giấy chứng nhận đủ điều kiện h...,Chúng tôi không tìm thấy thông tin liên quan.,Chúng tôi không tìm thấy thông tin liên quan.,"Chào bạn,\n\nTôi là iLaw, luật sư chuyên về sở..."
398,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Dịch vụ đánh giá công nghệ,09:00 | 25/10/2023,Mẫu Giấy chứng nhận đủ điều kiện hoạt động dịc...,Trình tự cấp Giấy chứng nhận đủ điều kiện hoạt...,Căn cứ quy định Điều 34Nghị định 76/2018/NĐ-CP...,Chúng tôi không tìm thấy thông tin liên quan.,Chúng tôi không tìm thấy thông tin liên quan.,"Chào bạn, tôi là iLaw, luật sư chuyên về sở hữ..."


## Validate by human

In [1]:
import pandas as pd
import ipywidgets as widgets
import textwrap
from IPython.display import display, clear_output

In [2]:
df = pd.read_csv("output_with_answers.csv")

In [3]:
df["score"] = ''

In [4]:
# State container for current index
state = {'current_index': 0}

def update_display(state):
    clear_output(wait=True)
    index = state['current_index']
    # In state hiện tại 
    print(f"Current index: {index}\n")
    
    print(f"Question: {df.loc[index, 'question']}")
    wrapped_content = textwrap.fill(df.loc[index, 'content'], width=200)
    print(f"Content:\n{wrapped_content}")
    
    # wrapped_answer = textwrap.fill(df.loc[index, 'answer'], width=200)
    # print(f"Answer: {wrapped_answer}")
    print(f"Answer: {df.loc[index, 'answer']}")

    # Handle None value in the 'score' column
    current_score = df.loc[index, 'score']
    if pd.isnull(current_score) or current_score not in [0, 1, 2, 3, 4, 5]:
        current_score = 0  # Set default score if None or invalid

    # Create score selection widget
    score_widget = widgets.Dropdown(
        options=[0, 1, 2, 3, 4, 5],
        value=current_score,
        description='Score:'
    )

    # Navigation buttons
    next_button = widgets.Button(description='Next')
    prev_button = widgets.Button(description='Previous')
    exit_button = widgets.Button(description='Exit')

    # Update score in the DataFrame when changed
    def on_score_change(change):
        df.loc[index, 'score'] = change.new

    score_widget.observe(on_score_change, names='value')

    # Navigation functions
    def on_next_clicked(b):
        if state['current_index'] < len(df) - 1:
            state['current_index'] += 1
            update_display(state)

    def on_prev_clicked(b):
        if state['current_index'] > 0:
            state['current_index'] -= 1
            update_display(state)

    def on_exit_clicked(b):
        clear_output(wait=True)
        print("Exiting... Final DataFrame:")
        display(df)

    # Attach event handlers to buttons
    next_button.on_click(on_next_clicked)
    prev_button.on_click(on_prev_clicked)
    exit_button.on_click(on_exit_clicked)

    # Display widgets
    display(score_widget, widgets.HBox([prev_button, next_button, exit_button]))

# Start the interactive session
update_display(state)


Current index: 0

Question: Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu có thể được bảo hộ trong bao lâu?
Content:
Căn cứ khoản 6 Điều 93Luật Sở hữu trí tuệ 2005quy định như sau: Điều 93. Hiệu lực của văn bằng bảo hộ 1. Văn bằng bảo hộ có hiệu lực trên toàn lãnh thổ Việt Nam. 2. Bằng độc quyền sáng chế có hiệu lực
từ ngày cấp và kéo dài đến hết hai mươi năm kể từ ngày nộp đơn. 3. Bằng độc quyền giải pháp hữu ích có hiệu lực từ ngày cấp và kéo dài đến hết mười năm kể từ ngày nộp đơn. 4. Bằng độc quyền kiểu dáng
công nghiệp có hiệu lực từ ngày cấp và kéo dài đến hết năm năm kể từ ngày nộp đơn, có thể gia hạn hai lần liên tiếp, mỗi lần năm năm. 5. Giấy chứng nhận đăng ký thiết kế bố trí mạch tích hợp bán dẫn
có hiệu lực từ ngày cấp và chấm dứt vào ngày sớm nhất trong số những ngày sau đây: a) Kết thúc mười năm kể từ ngày nộp đơn; b) Kết thúc mười năm kể từ ngày thiết kế bố trí được người có quyền đăng ký
hoặc người được người đó cho phép khai thác thương mại lần đầu tiên tại bất kỳ nơi n

Dropdown(description='Score:', options=(0, 1, 2, 3, 4, 5), value=0)

In [28]:
# nêu socre == '' thì gán bằng 0
df['score'] = df['score'].apply(lambda x: 0 if x == '' else x)

In [29]:
df

,link,label,date,title,question,content,answer,links,titles,score
0,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hiệu,23:00 | 27/11/2024,"Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu c...","Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu c...",Căn cứ khoản 6 Điều 93Luật Sở hữu trí tuệ 2005...,"Theo Luật Sở hữu trí tuệ Việt Nam, thời gian b...",https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Thông tư 01/2007/TT-BKHCN hướng dẫn thi hành N...,4
1,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hiệu,23:00 | 27/11/2024,"Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu c...",Thủ tục gia hạn Giấy chứng nhận đăng ký nhãn h...,Theo Điều 31Nghị định 65/2023/NĐ-CPquy định th...,Thủ tục gia hạn Giấy chứng nhận đăng ký nhãn h...,https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Quyết định 1291/QĐ-UBND năm 2018 công bố danh ...,3
2,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hiệu,23:00 | 27/11/2024,"Theo Luật Sở hữu trí tuệ Việt Nam, nhãn hiệu c...",Tải Mẫu Tờ khai yêu cầu gia hạn hiệu lực Giấy ...,Mẫu Tờ khai yêu cầu gia hạn hiệu lực Giấy chứn...,Để tải mẫu tờ khai yêu cầu gia hạn hiệu lực Gi...,https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Quyết định 3957/QĐ-UBND năm 2017 về quy chế qu...,3
3,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Quyền sở hữu công nghiệp,21:00 | 27/11/2024,Quyền sở hữu công nghiệp gồm các quyền nào?,Quyền sở hữu công nghiệp gồm các quyền nào?,Căn cứ khoản 4 Điều 4Luật Sở hữu trí tuệ 2005đ...,Quyền sở hữu công nghiệp là một phần quan trọn...,https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Quyết định 08/2008/QĐ-UBND quy định quản lý mộ...,5
4,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Quyền sở hữu công nghiệp,21:00 | 27/11/2024,Quyền sở hữu công nghiệp gồm các quyền nào?,Quyền sở hữu công nghiệp được xác lập như thế ...,Theo khoản 3 Điều 6Luật Sở hữu trí tuệ 2005đượ...,Quyền sở hữu công nghiệp tại Việt Nam được xác...,https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Quyết định 17/2010/QĐ-UBND ban hành Quy định h...,3
...,...,...,...,...,...,...,...,...,...,...
395,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hàng hóa,21:10 | 25/10/2023,Bao bì thương phẩm là gì? Bao bì dùng để đựng ...,Bao bì dùng để đựng hàng hóa bán lẻ có được xe...,Căn cứ quy định Điều 3Thông tư 05/2019/TT-BKHC...,Để trả lời câu hỏi của bạn về việc bao bì dùng...,https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Nghị định 105/2006/NĐ-CP Hướng dẫn Luật Sở hữu...,0
396,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Nhãn hàng hóa,21:10 | 25/10/2023,Bao bì thương phẩm là gì? Bao bì dùng để đựng ...,Vị trí nhãn hàng hóa đối với hàng hóa có cả ba...,Căn cứ quy định khoản 2 Điều 4Thông tư 05/2019...,"Theo pháp luật Việt Nam, vị trí nhãn hàng hóa ...",https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Quyết định 01/2008/QĐ-UBND về Quy chế quản lý ...,0
397,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Dịch vụ đánh giá công nghệ,09:00 | 25/10/2023,Mẫu Giấy chứng nhận đủ điều kiện hoạt động dịc...,Mẫu Giấy chứng nhận đủ điều kiện hoạt động dịc...,Dưới đây là mẫu Giấy chứng nhận đủ điều kiện h...,Chúng tôi không tìm thấy thông tin liên quan.,https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Thông tư 18/2011/TT-BKHCN sửa đổi Thông tư 01/...,0
398,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Dịch vụ đánh giá công nghệ,09:00 | 25/10/2023,Mẫu Giấy chứng nhận đủ điều kiện hoạt động dịc...,Trình tự cấp Giấy chứng nhận đủ điều kiện hoạt...,Căn cứ quy định Điều 34Nghị định 76/2018/NĐ-CP...,Chúng tôi không tìm thấy thông tin liên quan.,https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Quyết định 2472/QĐ-UBND năm 2023 công bố danh ...,0


In [30]:
df.to_csv("0_99.csv", index=False)